In [41]:
import re
import bs4
import time
import json
import requests
from bs4 import BeautifulSoup as bs

#### Funções

In [42]:
# Função para obter tudo que estiver nas tags td e div e com seletor CSS spaceit_pad
# Assim extraímos informações gerais da página
def getInfoGeral(soup):
    information = soup.select("td div .spaceit_pad")
    side_info_par = []
    for info in information:
        side_info_par.append(info.text.split())
    return side_info_par

In [43]:
# Função para obter o número de pessoas que adicionaram o anime aos favoritos
def getFavorite(information):
    for info in information:
        if info[0] == "Favorites:":
            data = info[1].split(",")
            return int("".join(data))
    return None

In [44]:
# Função para obter o número do episódio
def episodeNumberParser(information):
    for info in information:
        if (info[0] == "Episodes:"):
            if info[1] != "Unknown":
                return int(info[1])
            return None
    return None

In [45]:
# Função para obter o tipo de anime TV
def typeParser(information):
    for info in information:
        if (info[0] == "Type:"):
            return info[1]
    return None

In [46]:
# Função para obter o dia em que o anime estreou
def dayPremiered(information):
    for info in information:
        if (info[0] == "Premiered:"):
            premired_day = ""
            return " ".join(info[1:])
    return None

In [47]:
# Função para obter a fonte de inspiracao do anime
def getSource(information):
    for info in information:
        if info[0] == "Source:":
            return " ".join(info[1:])
    return None

In [48]:
# Função para retornar uma lista de gêneros do anime
def getGenres(information):
    for info in information:
        if info[0] == "Genres:":
            genres = "".join(info[1:])
            return genres.split(",")
    return None

In [49]:
# Função que retorna uma lista de estúdios que estão patrocinando este anime
def getStudios(information):
    for info in information:
        if info[0] == "Studios:":
            return " ".join(info[1:])
    return None

In [50]:
# Função que retorna uma lista de empresas que realmente possuem a licença deste anime
def getLicense(information):
    for info in information:
        if info[0] == "Licensors:":
            if info[1] == "None":
                return None
            license = "".join(info[1:])
            return license.split(",")
    return None

In [51]:
# Função que obtém a classificação do filme 
def getRating(information):
    for info in information:
        if info[0] == "Rating:":
            return "".join(info[1:4])
    return None

#### Encontarndo producers:

*finding_producers = soup2.find_all(["div","a"], class_=["spaceit_pad"])*

Primeiro encontre os dados usando find_all dos produtores mencionado acima.
Em seguida, enumere os dados encontrados.

*for index, j in enumerate(div_and_a):
    if finding_producers[index].text.split("\n")[1] == "Producers:":
        print(finding_producers[index])*

Se o primeiro índice == "Producers:"  então você obtém o índice onde encontrou os "Produtores".

E então usamos o índice conforme o código abaixo.

In [52]:
# Função para obter o nome de empresas que estão financiando o anime (Producers)
def producers_names(soup):
    div_and_a = soup.find_all(["div","a"], class_=["spaceit_pad"])
    for index, j in enumerate(div_and_a):
        if div_and_a[index].text.split("\n")[1] == "Producers:":
            res = re.sub(" *", "", div_and_a[index].text)
            new_res = res.split("\n")
            for i,j in enumerate(new_res):
                if j == "Producers:":
                    return new_res[i+1].split(",")
                    break
    return None

In [53]:
# Função para obter informações de ranking
def getRanking(soup):
    ranking = soup.select_one(".ranked strong").text
    data = ranking.split("#")[1]
    return int(data)

In [54]:
# Função para obter o score do Anime
def getScore(soup):
    data = float(soup.select_one(".score-label").text)
    return data

In [55]:
# Função para obter informação de popularidade
def getPopularity(soup):
    popularity = soup.select_one(".popularity strong").text
    data = popularity.split("#")[1]
    return int(data)

In [56]:
# Função para obter informação sobre os membros
def getMembers(soup):
    members = soup.select_one(".members strong").text
    data = members.split(",")
    return int("".join(data))

In [57]:
# Função para obter o número de pessoas que votaram no anime
def votersCount(soup):
    count = soup.select(".js-statistics-info span")
    data = []
    for i in count:
        data.append(i.text)
    if len(data) > 3:
        return int(data[2])
    return None

In [58]:
# Função para obter a voz do ator que faz a narração
def voiceActorsName(soup):
    actorNames = []
    names = soup.select(".js-anime-character-va-lang .spaceit_pad a")
    for name in names:
        data = "".join(name.text.split())
        actorNames.append(data)
    return actorNames

In [59]:
# Função para obter o idioma do ator que faz a narração
def voiceActorsLanguage(soup):
    languages = []
    langNames = soup.select(".js-anime-character-language")
    for language in langNames:
        lang = "".join(language.text.split())
        languages.append(lang)
    return languages

In [60]:
# Função que usa o voiceActorName e voiceActorsLanguage para criar um dataframe 
# que irá anexar todas as informações relacionadas às vozes dos atores.
# Usando essas informações, dividiremos os voiceActors com base em seus idiomas.
def voiceActorNameandLanguage(soup):
    names = voiceActorsName(soup)
    languages = voiceActorsLanguage(soup)
    namesAndLang = []
    for i in range(len(names)):
        namesAndLang.append([names[i], languages[i]])
    return namesAndLang

In [61]:
# Função que retorna a ocupação de cada funcionário
def staffOccupation(soup):
    data = []
    for i in soup.select('td .js-scrollfix-bottom-rel .spaceit_pad small'):
        data.append("".join(i.text.split()))
    return data

In [62]:
# Função que retorna o nome dos funcionários
def staffName(soup):
    data = []
    for i in soup.select('td .js-scrollfix-bottom-rel a'):
        data.append("".join(i.text.split()))
    uncleanData = []
    for index, info in enumerate(data):
        if info == "Addstaff":
            uncleanData.append(data[index+1:])
            break
    cleanData = []
    for i in uncleanData:
        for j in i:
            if len(j) > 1:
                cleanData.append(j)
    return cleanData

In [63]:
# Função que concatena nome e ocupação dos funcionários
def staffNameandOccupation(soup):
    name = staffName(soup)
    occupation = staffOccupation(soup)
    nameAndOccupation = []
    for i in range(len(name)):
        nameAndOccupation.append([name[i], occupation[i]])
    return nameAndOccupation

In [64]:
# Função que retorna o nome do Anime
def animeName(soup):
    name = soup.select_one(".title-name").text
    if len(name) > 1:
        return name
    return None

#### Web Scraping

In [83]:
for limit in range(0, 451, 50): 
    links = []
    url = f"https://myanimelist.net/topanime.php?limit={limit}"
    rq = requests.get(url)
    
    print(f"Requesting {url} - Status Code: {rq.status_code}")
    
    if rq.status_code != 200:
        print(f"Failed to retrieve {url}")
        continue
    
    soup = bs(rq.text, "html.parser")
    listsofA = soup.select("a.hoverinfo_trigger")
    
    for a in listsofA:
        href = a.get("href")
        if href and href.startswith("https://myanimelist.net/anime/"):
            links.append(href)
    
    if not links:
        print(f"No links found on {url}")
        continue
    
    with open(f"/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/AnimeLinks/animeLinks{limit}.txt", "w") as file:
        for link in links:
            file.write(link + '\n')
    
    time.sleep(10)

Requesting https://myanimelist.net/topanime.php?limit=0 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=50 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=100 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=150 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=200 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=250 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=300 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=350 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=400 - Status Code: 200
Requesting https://myanimelist.net/topanime.php?limit=450 - Status Code: 200


In [84]:
for limit in range(0, 451, 50):
    animeInfo = []
    
    with open(f"/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/AnimeLinks/animeLinks{limit}.txt", "r") as file:
        links = file.readlines()
    
    print("Fazendo Web Scraping. Seja paciente e aguarde...")
    
    for link in links:
        rq = requests.get(link.strip())
        rq2 = requests.get(link.strip() + "/characters")
        
        soup = bs(rq.text, "html.parser")
        soup2 = bs(rq2.text, "html.parser")
        
        information = getInfoGeral(soup)
        
        animeInfo.append(dict(
            name                      = animeName(soup),
            producers                 = producers_names(soup),
            ranking                   = getRanking(soup),
            votersCount               = votersCount(soup),
            score                     = getScore(soup),
            popularity                = getPopularity(soup),
            members                   = getMembers(soup),
            favorite                  = getFavorite(information),
            episodes                  = episodeNumberParser(information),
            animeType                 = typeParser(information),
            premiredDate              = dayPremiered(information),
            animeSource               = getSource(information),
            genres                    = getGenres(information),
            studios                   = getStudios(information),
            licenses                  = getLicense(information),
            rating                    = getRating(information),
            voiceActorNameandLanguage = voiceActorNameandLanguage(soup2),
            staffNameandOccupation    = staffNameandOccupation(soup2)
        ))
        
        time.sleep(10)

    with open(f"/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/AnimeInfo/animeInfo{limit}.txt", 'w') as fout:
        json.dump(animeInfo, fout)
    
    time.sleep(10)

Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...


In [85]:
import json
import pandas as pd

In [176]:
animeData = []

In [177]:
for i in range(0, 451, 50):
    with open(f"/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/AnimeInfo/animeInfo{i}.txt") as file:
        animeInfo = json.load(file)
        for entry in animeInfo:
            animeData.append(entry)

In [178]:
df1 = pd.DataFrame(animeData)

In [179]:
df1.shape

(1000, 18)

In [182]:
df1 = df1.drop_duplicates(subset=['name'])

In [183]:
df1.shape

(500, 18)

In [184]:
df1.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation
0,Sousou no Frieren,"[Aniplex, Dentsu, Shogakukan-ShueishaProductio...",1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,"[AdventureAdventure, DramaDrama, FantasyFantasy]",Madhouse,None,PG-13-Teens,"[[Ichinose,Kana, Japanese], [Harris,Jill, Engl...","[[Aoki,Haruka, Producer], [Fukushi,Yuuichirou,..."
2,Fullmetal Alchemist: Brotherhood,"[Aniplex, SquareEnix, MainichiBroadcastingSyst...",2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,"[ActionAction, AdventureAdventure, DramaDrama,...",Bones,"[Funimation, AniplexofAmerica]",R-17+,"[[Kugimiya,Rie, Japanese], [Ponticelli,Benedet...","[[Cook,Justin, Producer], [Maruyama,Hiroo, Pro..."
4,Steins;Gate,"[FrontierWorks, MediaFactory, KadokawaShoten, ...",3,1408020,9.07,13,2579543,190084,24.0,TV,Spring 2011,Visual novel,"[DramaDrama, Sci-FiSci-Fi, SuspenseSuspense]",White Fox,[Funimation],PG-13-Teens,"[[Seki,Tomokazu, Japanese], [Rinehart,Tyson, E...","[[Cook,Justin, Producer], [Danno,Yoshito, Prod..."
6,Gintama°,"[TVTokyo, Aniplex, Dentsu]",4,252993,9.06,342,635265,16713,51.0,TV,Spring 2015,Manga,"[ActionAction, ComedyComedy, Sci-FiSci-Fi]",Bandai Namco Pictures,"[Funimation, Crunchyroll]",PG-13-Teens,"[[Kugimiya,Rie, Japanese], [Ishii,Kouji, Japan...","[[Miyawaki,Chizuru, Director,Storyboard,KeyAni..."
8,Shingeki no Kyojin Season 3 Part 2,"[ProductionI.G, Dentsu, MainichiBroadcastingSy...",5,1601547,9.05,21,2298167,58957,10.0,TV,Spring 2019,Manga,"[ActionAction, DramaDrama, SuspenseSuspense]",Wit Studio,[Funimation],R-17+,"[[Ishikawa,Yui, Japanese], [Perino,Elena, Ital...","[[Cook,Justin, Producer], [Kinoshita,Tetsuya, ..."


In [185]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 998
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   name                       500 non-null    object 
 1   producers                  500 non-null    object 
 2   ranking                    500 non-null    int64  
 3   votersCount                500 non-null    int64  
 4   score                      500 non-null    float64
 5   popularity                 500 non-null    int64  
 6   members                    500 non-null    int64  
 7   favorite                   500 non-null    int64  
 8   episodes                   497 non-null    float64
 9   animeType                  500 non-null    object 
 10  premiredDate               301 non-null    object 
 11  animeSource                500 non-null    object 
 12  genres                     404 non-null    object 
 13  studios                    500 non-null    object 
 14 

In [186]:
df1.to_csv("/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/Processamento/animeInfoCombination.csv", index = False)

In [187]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [188]:
df = pd.read_csv("/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/Processamento/animeInfoCombination.csv")

In [189]:
pd.set_option("display.max_columns", 97)

In [190]:
df.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation
0,Sousou no Frieren,"['Aniplex', 'Dentsu', 'Shogakukan-ShueishaProd...",1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,"['AdventureAdventure', 'DramaDrama', 'FantasyF...",Madhouse,NaN,PG-13-Teens,"[['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...","[['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich..."
1,Fullmetal Alchemist: Brotherhood,"['Aniplex', 'SquareEnix', 'MainichiBroadcastin...",2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,"['ActionAction', 'AdventureAdventure', 'DramaD...",Bones,"['Funimation', 'AniplexofAmerica']",R-17+,"[['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...","[['Cook,Justin', 'Producer'], ['Maruyama,Hiroo..."
2,Steins;Gate,"['FrontierWorks', 'MediaFactory', 'KadokawaSho...",3,1408020,9.07,13,2579543,190084,24.0,TV,Spring 2011,Visual novel,"['DramaDrama', 'Sci-FiSci-Fi', 'SuspenseSuspen...",White Fox,['Funimation'],PG-13-Teens,"[['Seki,Tomokazu', 'Japanese'], ['Rinehart,Tys...","[['Cook,Justin', 'Producer'], ['Danno,Yoshito'..."
3,Gintama°,"['TVTokyo', 'Aniplex', 'Dentsu']",4,252993,9.06,342,635265,16713,51.0,TV,Spring 2015,Manga,"['ActionAction', 'ComedyComedy', 'Sci-FiSci-Fi']",Bandai Namco Pictures,"['Funimation', 'Crunchyroll']",PG-13-Teens,"[['Kugimiya,Rie', 'Japanese'], ['Ishii,Kouji',...","[['Miyawaki,Chizuru', 'Director,Storyboard,Key..."
4,Shingeki no Kyojin Season 3 Part 2,"['ProductionI.G', 'Dentsu', 'MainichiBroadcast...",5,1601547,9.05,21,2298167,58957,10.0,TV,Spring 2019,Manga,"['ActionAction', 'DramaDrama', 'SuspenseSuspen...",Wit Studio,['Funimation'],R-17+,"[['Ishikawa,Yui', 'Japanese'], ['Perino,Elena'...","[['Cook,Justin', 'Producer'], ['Kinoshita,Tets..."


In [191]:
df.shape

(500, 18)

In [192]:
df.producers

0      ['Aniplex', 'Dentsu', 'Shogakukan-ShueishaProd...
1      ['Aniplex', 'SquareEnix', 'MainichiBroadcastin...
2      ['FrontierWorks', 'MediaFactory', 'KadokawaSho...
3                       ['TVTokyo', 'Aniplex', 'Dentsu']
4      ['ProductionI.G', 'Dentsu', 'MainichiBroadcast...
                             ...                        
495                                       ['TwinEngine']
496    ['YomikoAdvertising', 'WarnerBros.Japan', 'Klo...
497    ['HalfH.PStudio', 'NipponColumbia', 'KlockWorx...
498    ['BandaiVisual', 'Dentsu', 'VictorEntertainmen...
499                   ['BandaiVisual', 'NihonAdSystems']
Name: producers, Length: 500, dtype: object

In [193]:
def remove_quote(word):
    return word.replace("'","")

In [194]:
def remove_openBracket(word):
    return word.replace("[","")

In [195]:
def remove_closeBracket(word):
    return word.replace("]","")

In [196]:
def remove_commaWithSemicolon(word):
    return word.replace(",",";")

In [197]:
def remove_space(word):
    return word.replace(" ","")

In [198]:
df.producers = remove_quote(df["producers"].str)
df.producers = remove_openBracket(df["producers"].str)
df.producers = remove_closeBracket(df["producers"].str)
df.producers = remove_commaWithSemicolon(df["producers"].str)
df.producers = remove_space(df["producers"].str)

In [199]:
df.producers

0      Aniplex;Dentsu;Shogakukan-ShueishaProductions;...
1      Aniplex;SquareEnix;MainichiBroadcastingSystem;...
2      FrontierWorks;MediaFactory;KadokawaShoten;Movi...
3                                 TVTokyo;Aniplex;Dentsu
4      ProductionI.G;Dentsu;MainichiBroadcastingSyste...
                             ...                        
495                                           TwinEngine
496    YomikoAdvertising;WarnerBros.Japan;KlockWorx;M...
497    HalfH.PStudio;NipponColumbia;KlockWorx;SonyMus...
498    BandaiVisual;Dentsu;VictorEntertainment;Nippon...
499                          BandaiVisual;NihonAdSystems
Name: producers, Length: 500, dtype: object

In [200]:
df.genres

0      ['AdventureAdventure', 'DramaDrama', 'FantasyF...
1      ['ActionAction', 'AdventureAdventure', 'DramaD...
2      ['DramaDrama', 'Sci-FiSci-Fi', 'SuspenseSuspen...
3       ['ActionAction', 'ComedyComedy', 'Sci-FiSci-Fi']
4      ['ActionAction', 'DramaDrama', 'SuspenseSuspen...
                             ...                        
495    ['ActionAction', 'AdventureAdventure', 'Fantas...
496    ['ActionAction', 'AdventureAdventure', 'Supern...
497    ['AdventureAdventure', 'ComedyComedy', 'Fantas...
498    ['ActionAction', 'AwardWinningAwardWinning', '...
499                                                  NaN
Name: genres, Length: 500, dtype: object

In [201]:
df.genres = remove_quote(df["genres"].str)
df.genres = remove_openBracket(df["genres"].str)
df.genres = remove_closeBracket(df["genres"].str)
df.genres = remove_commaWithSemicolon(df["genres"].str)
df.genres = remove_space(df["genres"].str)

In [202]:
df.genres

0           AdventureAdventure;DramaDrama;FantasyFantasy
1      ActionAction;AdventureAdventure;DramaDrama;Fan...
2               DramaDrama;Sci-FiSci-Fi;SuspenseSuspense
3                 ActionAction;ComedyComedy;Sci-FiSci-Fi
4               ActionAction;DramaDrama;SuspenseSuspense
                             ...                        
495       ActionAction;AdventureAdventure;FantasyFantasy
496    ActionAction;AdventureAdventure;SupernaturalSu...
497       AdventureAdventure;ComedyComedy;FantasyFantasy
498    ActionAction;AwardWinningAwardWinning;MysteryM...
499                                                  NaN
Name: genres, Length: 500, dtype: object

In [203]:
df.licenses

0                                                NaN
1                 ['Funimation', 'AniplexofAmerica']
2                                     ['Funimation']
3                      ['Funimation', 'Crunchyroll']
4                                     ['Funimation']
                           ...                      
495                                              NaN
496                                     ['VIZMedia']
497                 ['DiscotekMedia', 'Crunchyroll']
498    ['BandaiEntertainment', 'MangaEntertainment']
499                                              NaN
Name: licenses, Length: 500, dtype: object

In [204]:
df["licenses"] = remove_quote(df["licenses"].str)
df["licenses"] = remove_openBracket(df["licenses"].str)
df["licenses"] = remove_closeBracket(df["licenses"].str)
df["licenses"] = remove_commaWithSemicolon(df["licenses"].str)
df["licenses"] = remove_space(df["licenses"].str)

In [205]:
df.licenses

0                                         NaN
1                 Funimation;AniplexofAmerica
2                                  Funimation
3                      Funimation;Crunchyroll
4                                  Funimation
                        ...                  
495                                       NaN
496                                  VIZMedia
497                 DiscotekMedia;Crunchyroll
498    BandaiEntertainment;MangaEntertainment
499                                       NaN
Name: licenses, Length: 500, dtype: object

In [206]:
df.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation
0,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,AdventureAdventure;DramaDrama;FantasyFantasy,Madhouse,NaN,PG-13-Teens,"[['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...","[['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich..."
1,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,ActionAction;AdventureAdventure;DramaDrama;Fan...,Bones,Funimation;AniplexofAmerica,R-17+,"[['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...","[['Cook,Justin', 'Producer'], ['Maruyama,Hiroo..."
2,Steins;Gate,FrontierWorks;MediaFactory;KadokawaShoten;Movi...,3,1408020,9.07,13,2579543,190084,24.0,TV,Spring 2011,Visual novel,DramaDrama;Sci-FiSci-Fi;SuspenseSuspense,White Fox,Funimation,PG-13-Teens,"[['Seki,Tomokazu', 'Japanese'], ['Rinehart,Tys...","[['Cook,Justin', 'Producer'], ['Danno,Yoshito'..."
3,Gintama°,TVTokyo;Aniplex;Dentsu,4,252993,9.06,342,635265,16713,51.0,TV,Spring 2015,Manga,ActionAction;ComedyComedy;Sci-FiSci-Fi,Bandai Namco Pictures,Funimation;Crunchyroll,PG-13-Teens,"[['Kugimiya,Rie', 'Japanese'], ['Ishii,Kouji',...","[['Miyawaki,Chizuru', 'Director,Storyboard,Key..."
4,Shingeki no Kyojin Season 3 Part 2,ProductionI.G;Dentsu;MainichiBroadcastingSyste...,5,1601547,9.05,21,2298167,58957,10.0,TV,Spring 2019,Manga,ActionAction;DramaDrama;SuspenseSuspense,Wit Studio,Funimation,R-17+,"[['Ishikawa,Yui', 'Japanese'], ['Perino,Elena'...","[['Cook,Justin', 'Producer'], ['Kinoshita,Tets..."


In [207]:
newdf = pd.DataFrame(df.genres.str.split(';'))

In [208]:
newdf.head()

,genres
0,"[AdventureAdventure, DramaDrama, FantasyFantasy]"
1,"[ActionAction, AdventureAdventure, DramaDrama,..."
2,"[DramaDrama, Sci-FiSci-Fi, SuspenseSuspense]"
3,"[ActionAction, ComedyComedy, Sci-FiSci-Fi]"
4,"[ActionAction, DramaDrama, SuspenseSuspense]"


In [211]:
unique_values = []
for genres in newdf["genres"]:
    if type(genres) != float:
        for genre in genres:
            if genre not in unique_values:
                unique_values.append(genre)

In [212]:
unique_values

['AdventureAdventure',
 'DramaDrama',
 'FantasyFantasy',
 'ActionAction',
 'Sci-FiSci-Fi',
 'SuspenseSuspense',
 'ComedyComedy',
 'RomanceRomance',
 'SupernaturalSupernatural',
 'AwardWinningAwardWinning',
 'MysteryMystery',
 'SportsSports',
 'SliceofLifeSliceofLife',
 'EcchiEcchi',
 'GourmetGourmet',
 'HorrorHorror',
 'AvantGardeAvantGarde',
 'BoysLoveBoysLove']

In [213]:
for index, genres in enumerate(newdf.genres):
    new_genre = ""
    if type(genres) != float:
        for genre in genres:
            if genre == "ActionAction":
                new_genre += "Action"
            elif genre == "AdventureAdventure":
                new_genre += "Adventure"
            elif genre == "ComedyComedy":
                new_genre += "Comedy"
            elif genre == "DramaDrama":
                new_genre += "Drama"
            elif genre == "FantasyFantasy":
                new_genre += "Fantasy"
            elif genre == "Sci-FiSci-Fi":
                new_genre += "Sci-fi"
            elif genre == "MysteryMystery":
                new_genre += "Mystery"
            elif genre == "SuspenseSuspense":
                new_genre += "Suspense"
            elif genre == "RomanceRomance":
                new_genre += "Romance"
            elif genre == "SliceofLifeSliceofLife":
                new_genre += "Sliceoflife"
            elif genre == "SupernaturalSupernatural":
                new_genre += "Supernatural"
            elif genre == "SportsSports":
                new_genre += "Sports"
            elif genre == "HorrorHorror":
                new_genre += "Horror"
            elif genre == "EcchiEcchi":
                new_genre += "Ecchi"
            elif genre == "AvantGardeAvantGarde":
                new_genre += "Avant_garde"
            elif genre == "BoysLoveBoysLove":
                new_genre += "Boys_love"
            elif genre == "GourmetGourmet":
                new_genre += "Gourmet"
            elif genre == "GirlsLoveGirlsLove":
                new_genre += "Girls_love"
            elif genre == "WorkLifeWorkLife":
                new_genre += "Work_life"
    newdf["genres"][index] = new_genre

In [214]:
for index, genres in enumerate(newdf.genres):
    if type(genres) != float:
        newdf["genres"][index] = ";".join(re.findall('[A-Z][^A-Z]*', newdf["genres"][index]))

In [215]:
newdf.head()

,genres
0,Adventure;Drama;Fantasy
1,Action;Adventure;Drama;Fantasy
2,Drama;Sci-fi;Suspense
3,Action;Comedy;Sci-fi
4,Action;Drama;Suspense


In [216]:
df["genres"] = newdf["genres"]

In [217]:
df.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation
0,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"[['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...","[['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich..."
1,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"[['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...","[['Cook,Justin', 'Producer'], ['Maruyama,Hiroo..."
2,Steins;Gate,FrontierWorks;MediaFactory;KadokawaShoten;Movi...,3,1408020,9.07,13,2579543,190084,24.0,TV,Spring 2011,Visual novel,Drama;Sci-fi;Suspense,White Fox,Funimation,PG-13-Teens,"[['Seki,Tomokazu', 'Japanese'], ['Rinehart,Tys...","[['Cook,Justin', 'Producer'], ['Danno,Yoshito'..."
3,Gintama°,TVTokyo;Aniplex;Dentsu,4,252993,9.06,342,635265,16713,51.0,TV,Spring 2015,Manga,Action;Comedy;Sci-fi,Bandai Namco Pictures,Funimation;Crunchyroll,PG-13-Teens,"[['Kugimiya,Rie', 'Japanese'], ['Ishii,Kouji',...","[['Miyawaki,Chizuru', 'Director,Storyboard,Key..."
4,Shingeki no Kyojin Season 3 Part 2,ProductionI.G;Dentsu;MainichiBroadcastingSyste...,5,1601547,9.05,21,2298167,58957,10.0,TV,Spring 2019,Manga,Action;Drama;Suspense,Wit Studio,Funimation,R-17+,"[['Ishikawa,Yui', 'Japanese'], ['Perino,Elena'...","[['Cook,Justin', 'Producer'], ['Kinoshita,Tets..."


In [219]:
df.voiceActorNameandLanguage

0      [['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...
1      [['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...
2      [['Seki,Tomokazu', 'Japanese'], ['Rinehart,Tys...
3      [['Kugimiya,Rie', 'Japanese'], ['Ishii,Kouji',...
4      [['Ishikawa,Yui', 'Japanese'], ['Perino,Elena'...
                             ...                        
495    [['Kobayashi,Chiaki', 'Japanese'], ['Saab,Alej...
496    [['Miyake,Kenta', 'Japanese'], ['Tergliafera,C...
497    [['Amamiya,Sora', 'Japanese'], ['Mata,Faye', '...
498    [['Saka,Osamu', 'Japanese'], ['Knight,WilliamF...
499    [['Kamiya,Hiroshi', 'Japanese'], ['Suwabe,Juni...
Name: voiceActorNameandLanguage, Length: 500, dtype: object

In [218]:
newdf = pd.DataFrame(df.voiceActorNameandLanguage)

In [225]:
for index, data in enumerate(newdf["voiceActorNameandLanguage"]):
    if type(data) != float:
        newdf["voiceActorNameandLanguage"][index] = newdf["voiceActorNameandLanguage"][index].replace("[[","[")
        newdf["voiceActorNameandLanguage"][index] = newdf["voiceActorNameandLanguage"][index].replace("]]","]")

In [226]:
newdf["voiceActorNameandLanguage"] = remove_quote(newdf["voiceActorNameandLanguage"].str)
newdf["voiceActorNameandLanguage"] = remove_openBracket(newdf["voiceActorNameandLanguage"].str)

In [227]:
newdf.voiceActorNameandLanguage[0]

'Ichinose,Kana, Japanese], Harris,Jill, English], Thellier,Jehanne, French], Rosis,MariaClara, Portuguese(BR)], Marteddu,Agnese, Italian], Estrada,Azucena, Spanish], Fölster,Linda, German], Tanezaki,Atsumi, Japanese], Rodak,Mallorie, English], Nonnenmacher,Marie, French], Souza,Jacque, Portuguese(BR)], Felli,Martina, Italian], Ugalde,Erika, Spanish], Casper,Julia, German], Kobayashi,Chiaki, Japanese], Cruz,JordanDash, English], Kiyoto,Arisa, Japanese], Miagusuku,Lucas, Portuguese(BR)], Lecordier,Brigitte, French], Marteddu,Tito, Italian], MartínezTorizes,Eduardo, Spanish], Karbowski,Brittney, English], Blanc,Kelyan, French], Schächter,Janek, German], Taketatsu,Ayana, Japanese], Pettit,Corey, English], Bienaimé,Nathalie, French], Russello,Federica, Italian], Horta,Luísa, Portuguese(BR)], Olea,Vanessa, Spanish], deLuis,Noelia, Spanish], Takahashi,Shinya, Japanese], Kamui,Daniel, Portuguese(BR)], Franco,Alberto, Italian], Mukai,Brent, English], Méyère,Bruno, French], Kohara,Konomi, Japane

In [228]:
# Remove "], " e então [ e então faz o split dos dados com base em ; 
# Por fim, criar a lista de atores e sseus idiomas
for index, data in enumerate(newdf.voiceActorNameandLanguage):
    if type(data) != float:
        newdf["voiceActorNameandLanguage"][index] = re.sub(r"], ", ";", newdf["voiceActorNameandLanguage"][index])
        newdf["voiceActorNameandLanguage"][index] = remove_closeBracket(newdf["voiceActorNameandLanguage"][index])
        newdf["voiceActorNameandLanguage"][index] = newdf["voiceActorNameandLanguage"][index].split(";")
        for i, newData in enumerate(newdf["voiceActorNameandLanguage"][index]):
            newdf["voiceActorNameandLanguage"][index][i] = newdf["voiceActorNameandLanguage"][index][i].split(", ")

In [229]:
newdf.voiceActorNameandLanguage[0]

[['Ichinose,Kana', 'Japanese'],
 ['Harris,Jill', 'English'],
 ['Thellier,Jehanne', 'French'],
 ['Rosis,MariaClara', 'Portuguese(BR)'],
 ['Marteddu,Agnese', 'Italian'],
 ['Estrada,Azucena', 'Spanish'],
 ['Fölster,Linda', 'German'],
 ['Tanezaki,Atsumi', 'Japanese'],
 ['Rodak,Mallorie', 'English'],
 ['Nonnenmacher,Marie', 'French'],
 ['Souza,Jacque', 'Portuguese(BR)'],
 ['Felli,Martina', 'Italian'],
 ['Ugalde,Erika', 'Spanish'],
 ['Casper,Julia', 'German'],
 ['Kobayashi,Chiaki', 'Japanese'],
 ['Cruz,JordanDash', 'English'],
 ['Kiyoto,Arisa', 'Japanese'],
 ['Miagusuku,Lucas', 'Portuguese(BR)'],
 ['Lecordier,Brigitte', 'French'],
 ['Marteddu,Tito', 'Italian'],
 ['MartínezTorizes,Eduardo', 'Spanish'],
 ['Karbowski,Brittney', 'English'],
 ['Blanc,Kelyan', 'French'],
 ['Schächter,Janek', 'German'],
 ['Taketatsu,Ayana', 'Japanese'],
 ['Pettit,Corey', 'English'],
 ['Bienaimé,Nathalie', 'French'],
 ['Russello,Federica', 'Italian'],
 ['Horta,Luísa', 'Portuguese(BR)'],
 ['Olea,Vanessa', 'Spanish'],

In [231]:
unique_languages = []
for index, info in enumerate(newdf.voiceActorNameandLanguage):
    for data in newdf["voiceActorNameandLanguage"][index]:
        if len(data) > 1:
            if data[1] not in unique_languages:
                unique_languages.append(data[1])

In [232]:
unique_languages

['Japanese',
 'English',
 'French',
 'Portuguese(BR)',
 'Italian',
 'Spanish',
 'German',
 'Hungarian',
 'Korean',
 'Mandarin',
 'Hebrew']

In [233]:
# Criando um dicionário de lista para que possamos transformá-lo em um dataframe mais tarde
new_dic = dict()
for data in unique_languages:
    new_dic[data] = []

In [234]:
new_dic

{'Japanese': [],
 'English': [],
 'French': [],
 'Portuguese(BR)': [],
 'Italian': [],
 'Spanish': [],
 'German': [],
 'Hungarian': [],
 'Korean': [],
 'Mandarin': [],
 'Hebrew': []}

In [236]:
for index, info in enumerate(newdf.voiceActorNameandLanguage):
    if len(info) > 1:
        Japanese = []
        Italian = []
        PortugueseBR = []
        German = []
        Hungarian = []
        English = []
        Korean = []
        Spanish = []
        French = []
        Mandarin = []
        Hebrew = []
        for data in newdf["voiceActorNameandLanguage"][index]:
            if data[1] == "Japanese":
                Japanese.append(data[0])
            elif data[1] == "Italian":
                Italian.append(data[0])
            elif data[1] == "Portuguese(BR)":
                PortugueseBR.append(data[0])
            elif data[1] == "German":
                German.append(data[0])
            elif data[1] == "Hungarian":
                Hungarian.append(data[0])
            elif data[1] == "English":
                English.append(data[0])
            elif data[1] == "Korean":
                Korean.append(data[0])
            elif data[1] == "Spanish":
                Spanish.append(data[0])
            elif data[1] == "French":
                French.append(data[0])
            elif data[1] == "Mandarin":
                Mandarin.append(data[0])
            elif data[1] == "Hebrew":
                Hebrew.append(data[0])
        new_dic["Japanese"].append(";".join(Japanese))
        new_dic["Italian"].append(";".join(Italian))
        new_dic["Portuguese(BR)"].append(";".join(PortugueseBR))
        new_dic["German"].append(";".join(German))
        new_dic["Hungarian"].append(";".join(Hungarian))
        new_dic["English"].append(";".join(English))
        new_dic["Korean"].append(";".join(Korean))
        new_dic["Spanish"].append(";".join(Spanish))
        new_dic["French"].append(";".join(French))
        new_dic["Mandarin"].append(";".join(Mandarin))
        new_dic["Hebrew"].append(";".join(Hebrew))
    else:
        new_dic["Japanese"].append(np.nan)
        new_dic["Italian"].append(np.nan)
        new_dic["Portuguese(BR)"].append(np.nan)
        new_dic["German"].append(np.nan)
        new_dic["Hungarian"].append(np.nan)
        new_dic["English"].append(np.nan)
        new_dic["Korean"].append(np.nan)
        new_dic["Spanish"].append(np.nan)
        new_dic["French"].append(np.nan)
        new_dic["Mandarin"].append(np.nan)
        new_dic["Hebrew"].append(np.nan)

In [237]:
newestdf = pd.DataFrame(new_dic)

In [238]:
newestdf.head()

,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew
0,"Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",,,,
1,"Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",,
2,"Seki,Tomokazu;Imai,Asami;Miyano,Mamoru;Hanazaw...","Rinehart,Tyson;Nishimura,Trina;Tatum,J.Michael...","Pasquier,Laurent;Cornibert,Sarah;Wilhelm,Marc;...","Casemiro,Bruno;Cunha,Maitê;Rinaldi,André;Lua,B...","Vivio,Paolo;Paolelli,Rachele;Chevalier,Davide;...","Fernández,Iván;Vázquez,Camila;Naro,Jordi;Ramír...","Wirthgen,Jesco;Bäcker,Manuela;Gavrilis,Marios;...","Seszták,Szabolcs;NemesTakách,Kata;Juhász,Zoltá...","Wu,JeongSin;Hong,SiHo;Yeo,MinJeong;Kim,SeoYeon...",,
3,"Kugimiya,Rie;Ishii,Kouji;Sugita,Tomokazu;Tomat...","Loewen,Jocelyne;Daingerfield,Michael;Pitre,Sab...",,,"Granato,Jolanda;Fazio,Davide;Viola,Federico;Ma...",,,,,,
4,"Ishikawa,Yui;Inoue,Marina;Kamiya,Hiroshi;Kaji,...","Nishimura,Trina;Grelle,JessieJames;Mercer,Matt...","Bienaimé,Nathalie;Niverd,Victor;Méyère,Bruno;B...","Stefane,Mayara;Volpato,Pedro;Sangregório,Bruno...","Perino,Elena;Bolognesi,Gaia;Raffaeli,Daniele;C...","Molina,Graciela;Lobo,Ana;Gómez,Marc;deAlba,Héc...","Felder,Max;Gavrilis,Marios;Schwarzmaier,Tim;Da...",,,,


In [239]:
df = df.merge(newestdf, left_index = True, right_index = True, sort = False)

In [240]:
df.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew
0,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"[['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...","[['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich...","Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",,,,
1,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"[['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...","[['Cook,Justin', 'Producer'], ['Maruyama,Hiroo...","Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",,
2,Steins;Gate,FrontierWorks;MediaFactory;KadokawaShoten;Movi...,3,1408020,9.07,13,2579543,190084,24.0,TV,Spring 2011,Visual novel,Drama;Sci-fi;Suspense,White Fox,Funimation,PG-13-Teens,"[['Seki,Tomokazu', 'Japanese'], ['Rinehart,Tys...","[['Cook,Justin', 'Producer'], ['Danno,Yoshito'...","Seki,Tomokazu;Imai,Asami;Miyano,Mamoru;Hanazaw...","Rinehart,Tyson;Nishimura,Trina;Tatum,J.Michael...","Pasquier,Laurent;Cornibert,Sarah;Wilhelm,Marc;...","Casemiro,Bruno;Cunha,Maitê;Rinaldi,André;Lua,B...","Vivio,Paolo;Paolelli,Rachele;Chevalier,Davide;...","Fernández,Iván;Vázquez,Camila;Naro,Jordi;Ramír...","Wirthgen,Jesco;Bäcker,Manuela;Gavrilis,Marios;...","Seszták,Szabolcs;NemesTakách,Kata;Juhász,Zoltá...","Wu,JeongSin;Hong,SiHo;Yeo,MinJeong;Kim,SeoYeon...",,
3,Gintama°,TVTokyo;Aniplex;Dentsu,4,252993,9.06,342,635265,16713,51.0,TV,Spring 2015,Manga,Action;Comedy;Sci-fi,Bandai Namco Pictures,Funimation;Crunchyroll,PG-13-Teens,"[['Kugimiya,Rie', 'Japanese'], ['Ishii,Kouji',...","[['Miyawaki,Chizuru', 'Director,Storyboard,Key...","Kugimiya,Rie;Ishii,Kouji;Sugita,Tomokazu;Tomat...","Loewen,Jocelyne;Daingerfield,Michael;Pitre,Sab...",,,"Granato,Jolanda;Fazio,Davide;Viola,Federico;Ma...",,,,,,
4,Shingeki no Kyojin Season 3 Part 2,ProductionI.G;Dentsu;MainichiBroadcastingSyste...,5,1601547,9.05,21,2298167,58957,10.0,TV,Spring 2019,Manga,Action;Drama;Suspense,Wit Studio,Funimation,R-17+,"[['Ishikawa,Yui', 'Japanese'], ['Perino,Elena'...","[['Cook,Justin', 'Producer'], ['Kinoshita,Tets...","Ishikawa,Yui;Inoue,Marina;Kamiya,Hiroshi;Kaji,...","Nishimura,Trina;Grelle,JessieJames;Mercer,Matt...","Bienaimé,Nathalie;Niverd,Victor;Méyère,Bruno;B...","Stefane,Mayara;Volpato,Pedro;Sangregório,Bruno...","Perino,Elena;Bolognesi,Gaia;Raffaeli,Daniele;C...","Molina,Graciela;Lobo,Ana;Gómez,Marc;deAlba,Héc...","Felder,Max;Gavrilis,Marios;Schwarzmaier,Tim;Da...",,,,


In [242]:
df.staffNameandOccupation

0      [['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich...
1      [['Cook,Justin', 'Producer'], ['Maruyama,Hiroo...
2      [['Cook,Justin', 'Producer'], ['Danno,Yoshito'...
3      [['Miyawaki,Chizuru', 'Director,Storyboard,Key...
4      [['Cook,Justin', 'Producer'], ['Kinoshita,Tets...
                             ...                        
495    [['Ishii,Nozomi', 'Producer'], ['Kawagoe,Watar...
496    [['Fukuda,Jun', 'Producer'], ['Kajita,Kouji', ...
497    [['Ogura,Rie', 'Producer'], ['Urasaki,Nobumits...
498    [['Kunisaki,Hisanori', 'Producer'], ['Maseba,Y...
499    [['Morihiro,Fumi', 'Producer'], ['Fujimaki,Tad...
Name: staffNameandOccupation, Length: 500, dtype: object

In [243]:
staffNnO = pd.DataFrame(df["staffNameandOccupation"])

In [244]:
for index, data in enumerate(staffNnO["staffNameandOccupation"]):
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("[[", "[")
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("]]", "]")
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("[\'", "")
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("\'", "")
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("], ", ";")
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].replace("]", "")

In [246]:
staffNnO.staffNameandOccupation[0]

'Aoki,Haruka, Producer;Fukushi,Yuuichirou, Producer;Itou,Yuukou, Producer;Iwasa,Naoki, Producer;Kikuchi,Ruriko, Producer;Nakanome,Takashi, Producer;Shikama,Taisuke, Producer;Taguchi,Ayuri, Producer;Taguchi,Shouichirou, Producer;Saitou,Keiichirou, Director,EpisodeDirector,Storyboard,KeyAnimation,2ndKeyAnimation;Hata,Shouji, SoundDirector;Andou,Ken, EpisodeDirector;Asaka,Morio, EpisodeDirector;Fujii,Kunio, EpisodeDirector;Fujimoto,Kouki, EpisodeDirector,Storyboard,KeyAnimation;Harashina,Daiki, EpisodeDirector,Storyboard,AnimationDirector;Iino,Shinya, EpisodeDirector,Storyboard;Im,Ga-Hee, EpisodeDirector;Isogawa,Kazumasa, EpisodeDirector;Iwazawa,Tooru, EpisodeDirector,Storyboard,KeyAnimation;Kakita,Hayato, EpisodeDirector,2ndKeyAnimation,KeyAnimation;Kariya,Nobuhide, EpisodeDirector,Storyboard,AnimationDirector,KeyAnimation;Kawasaki,Yoshiki, EpisodeDirector,Storyboard;Kimura,Taku, EpisodeDirector;Kitagawa,Tomoya, EpisodeDirector,Storyboard;Kojima,Keisuke, EpisodeDirector,Storyboard,KeyAni

In [247]:
for index, data in enumerate(staffNnO["staffNameandOccupation"]):
    staffNnO["staffNameandOccupation"][index] = staffNnO["staffNameandOccupation"][index].split(";")
    for i, info in enumerate(staffNnO["staffNameandOccupation"][index]):
        staffNnO["staffNameandOccupation"][index][i] = staffNnO["staffNameandOccupation"][index][i].split(", ")

In [250]:
staffNnO.staffNameandOccupation[0]

[['Aoki,Haruka', 'Producer'],
 ['Fukushi,Yuuichirou', 'Producer'],
 ['Itou,Yuukou', 'Producer'],
 ['Iwasa,Naoki', 'Producer'],
 ['Kikuchi,Ruriko', 'Producer'],
 ['Nakanome,Takashi', 'Producer'],
 ['Shikama,Taisuke', 'Producer'],
 ['Taguchi,Ayuri', 'Producer'],
 ['Taguchi,Shouichirou', 'Producer'],
 ['Saitou,Keiichirou',
  'Director,EpisodeDirector,Storyboard,KeyAnimation,2ndKeyAnimation'],
 ['Hata,Shouji', 'SoundDirector'],
 ['Andou,Ken', 'EpisodeDirector'],
 ['Asaka,Morio', 'EpisodeDirector'],
 ['Fujii,Kunio', 'EpisodeDirector'],
 ['Fujimoto,Kouki', 'EpisodeDirector,Storyboard,KeyAnimation'],
 ['Harashina,Daiki', 'EpisodeDirector,Storyboard,AnimationDirector'],
 ['Iino,Shinya', 'EpisodeDirector,Storyboard'],
 ['Im,Ga-Hee', 'EpisodeDirector'],
 ['Isogawa,Kazumasa', 'EpisodeDirector'],
 ['Iwazawa,Tooru', 'EpisodeDirector,Storyboard,KeyAnimation'],
 ['Kakita,Hayato', 'EpisodeDirector,2ndKeyAnimation,KeyAnimation'],
 ['Kariya,Nobuhide',
  'EpisodeDirector,Storyboard,AnimationDirector,KeyA

In [251]:
for i in range(len(staffNnO["staffNameandOccupation"])):
    for j, job in enumerate(staffNnO["staffNameandOccupation"][i]):
        if len(staffNnO["staffNameandOccupation"][i][j]) > 1:
            staffNnO["staffNameandOccupation"][i][j][-1] = staffNnO["staffNameandOccupation"][i][j][1].split(",")

In [252]:
staffNnO

,staffNameandOccupation
0,"[[Aoki,Haruka, [Producer]], [Fukushi,Yuuichiro..."
1,"[[Cook,Justin, [Producer]], [Maruyama,Hiroo, [..."
2,"[[Cook,Justin, [Producer]], [Danno,Yoshito, [P..."
3,"[[Miyawaki,Chizuru, [Director, Storyboard, Key..."
4,"[[Cook,Justin, [Producer]], [Kinoshita,Tetsuya..."
...,...
495,"[[Ishii,Nozomi, [Producer]], [Kawagoe,Wataru, ..."
496,"[[Fukuda,Jun, [Producer]], [Kajita,Kouji, [Pro..."
497,"[[Ogura,Rie, [Producer]], [Urasaki,Nobumitsu, ..."
498,"[[Kunisaki,Hisanori, [Producer]], [Maseba,Yuta..."


In [253]:
unique_profes = []
for i in range(len(staffNnO["staffNameandOccupation"])):
    for j, job in enumerate(staffNnO["staffNameandOccupation"][i]):
        if len(job) > 1:
            for k, profes in enumerate(job[1]):
                if profes not in unique_profes:
                    unique_profes.append(profes)

In [254]:
unique_profes

['Producer',
 'Director',
 'EpisodeDirector',
 'Storyboard',
 'KeyAnimation',
 '2ndKeyAnimation',
 'SoundDirector',
 'AnimationDirector',
 'In-BetweenAnimation',
 'Script',
 'SeriesComposition',
 'ThemeSongComposition',
 'ThemeSongLyrics',
 'ThemeSongArrangement',
 'Music',
 'ThemeSongPerformance',
 'InsertedSongPerformance',
 'OriginalCharacterDesign',
 'AssociateProducer',
 'ExecutiveProducer',
 'ChiefAnimationDirector',
 'SpecialEffects',
 'DirectorofPhotography',
 'Planning',
 'SoundEffects',
 'AnimationCheck',
 'Editing',
 'AssistantAnimationDirector',
 'ADRDirector',
 'CharacterDesign',
 'ColorDesign',
 'BackgroundArt',
 'ArtDirector',
 'ChiefProducer',
 'OriginalCreator',
 'Publicity',
 'DigitalPaint',
 'ProductionCoordination',
 'AssistantProducer',
 'DialogueEditing',
 'ColorSetting',
 'AssistantDirector',
 'Setting',
 'RecordingEngineer',
 'PlanningProducer',
 'Screenplay',
 'Layout',
 'ProductionManager',
 'ProductionAssistant',
 'AssistantProductionCoordinat',
 'CastingDire

In [283]:
profes_dic = dict()
for data in unique_profes:
    profes_dic[data] = []

In [284]:
len(profes_dic)

65

In [285]:
print(profes_dic)

{'Producer': [], 'Director': [], 'EpisodeDirector': [], 'Storyboard': [], 'KeyAnimation': [], '2ndKeyAnimation': [], 'SoundDirector': [], 'AnimationDirector': [], 'In-BetweenAnimation': [], 'Script': [], 'SeriesComposition': [], 'ThemeSongComposition': [], 'ThemeSongLyrics': [], 'ThemeSongArrangement': [], 'Music': [], 'ThemeSongPerformance': [], 'InsertedSongPerformance': [], 'OriginalCharacterDesign': [], 'AssociateProducer': [], 'ExecutiveProducer': [], 'ChiefAnimationDirector': [], 'SpecialEffects': [], 'DirectorofPhotography': [], 'Planning': [], 'SoundEffects': [], 'AnimationCheck': [], 'Editing': [], 'AssistantAnimationDirector': [], 'ADRDirector': [], 'CharacterDesign': [], 'ColorDesign': [], 'BackgroundArt': [], 'ArtDirector': [], 'ChiefProducer': [], 'OriginalCreator': [], 'Publicity': [], 'DigitalPaint': [], 'ProductionCoordination': [], 'AssistantProducer': [], 'DialogueEditing': [], 'ColorSetting': [], 'AssistantDirector': [], 'Setting': [], 'RecordingEngineer': [], 'Plann

In [299]:
profes_dic = {
    "Producer": [],
    "Director": [],
    "EpisodeDirector": [],
    "Storyboard": [],
    "SoundDirector": [],
    "KeyAnimation": [],
    "2ndKeyAnimation": [],
    "CharacterDesign": [],
    "Script": [],
    "SeriesComposition": [],
    "ThemeSongLyrics": [],
    "ThemeSongPerformance": [],
    "ThemeSongArrangement": [],
    "ThemeSongComposition": [],
    "AnimationDirector": [],
    "OriginalCreator": [],
    "ADRDirector": [],
    "ExecutiveProducer": [],
    "ChiefAnimationDirector": [],
    "DirectorofPhotography": [],
    "In-BetweenAnimation": [],
    "BackgroundArt": [],
    "Planning": [],
    "ColorDesign": [],
    "AssistantAnimationDirector": [],
    "Editing": [],
    "ArtDirector": [],
    "Music": [],
    "ProductionCoordination": [],
    "DigitalPaint": [],
    "ColorSetting": [],
    "AssistantDirector": [],
    "SoundEffects": [],
    "InsertedSongPerformance": [],
    "OriginalCharacterDesign": [],
    "Setting": [],
    "Screenplay": [],
    "AssociateProducer": [],
    "SpecialEffects": [],
    "AnimationCheck": [],
    "ProductionAssistant": [],
    "SoundSupervisor": [],
    "PrincipleDrawing": [],
    "AssistantProducer": [],
    "ProductionManager": [],
    "SoundManager": [],
    "Co-Director": [],
    "SeriesProductionDirector": [],
    "MechanicalDesign": [],
    "PlanningProducer": [],
    "Layout": [],
    "ChiefProducer": [],
    "Recording": [],
    "DialogueEditing": [],
    "RecordingEngineer": [],
    "Creator": [],
    "RecordingAssistant": [],
    "Publicity": [],
    "Co-Producer": [],
    "CastingDirector": [],
    "Re-RecordingMixing": [],
    "AssistantProductionCoordinat": [],
    "SettingManager": []
}

for i in range(len(staffNnO["staffNameandOccupation"])):
    Producer = []
    Director = []
    EpisodeDirector = []
    Storyboard = []
    SoundDirector = []
    KeyAnimation = []
    KeyAnimation2nd = []
    CharacterDesign = []
    Script = []
    SeriesComposition = []
    ThemeSongLyrics = []
    ThemeSongPerformance = []
    ThemeSongArrangement = []
    ThemeSongComposition = []
    AnimationDirector = []
    OriginalCreator = []
    ADRDirector = []
    ExecutiveProducer = []
    ChiefAnimationDirector = []
    DirectorofPhotography = []
    In_BetweenAnimation = []
    BackgroundArt = []
    Planning = []
    ColorDesign = []
    AssistantAnimationDirector = []
    Editing = []
    ArtDirector = []
    Music = []
    ProductionCoordination = []
    DigitalPaint = []
    ColorSetting = []
    AssistantDirector = []
    SoundEffects = []
    InsertedSongPerformance = []
    OriginalCharacterDesign = []
    Setting = []
    Screenplay = []
    AssociateProducer = []
    SpecialEffects = []
    AnimationCheck = []
    ProductionAssistant = []
    SoundSupervisor = []
    PrincipleDrawing = []
    AssistantProducer = []
    ProductionManager = []
    SoundManager = []
    Co_Director = []
    SeriesProductionDirector = []
    MechanicalDesign = []
    PlanningProducer = []
    Layout = []
    ChiefProducer = []
    Recording = []
    DialogueEditing = []
    RecordingEngineer = []
    Creator = []
    RecordingAssistant = []
    Publicity = []
    Co_Producer = []
    CastingDirector = []
    Re_RecordingMixing = []
    AssistantProductionCoordinat = []
    SettingManager = []
    
    for job in staffNnO["staffNameandOccupation"][i]:
        if len(job) > 1:
            for profes in job[1]:
                if profes == "Producer":
                    Producer.append(job[0])
                elif profes == 'Director':
                    Director.append(job[0])
                elif profes == 'EpisodeDirector':
                    EpisodeDirector.append(job[0])
                elif profes == 'Storyboard':
                    Storyboard.append(job[0])
                elif profes == 'SoundDirector':
                    SoundDirector.append(job[0])
                elif profes == 'KeyAnimation':
                    KeyAnimation.append(job[0])
                elif profes == '2ndKeyAnimation':
                    KeyAnimation2nd.append(job[0])
                elif profes == 'CharacterDesign':
                    CharacterDesign.append(job[0])
                elif profes == 'Script':
                    Script.append(job[0])
                elif profes == 'SeriesComposition':
                    SeriesComposition.append(job[0])
                elif profes == 'ThemeSongLyrics':
                    ThemeSongLyrics.append(job[0])
                elif profes == 'ThemeSongPerformance':
                    ThemeSongPerformance.append(job[0])
                elif profes == 'ThemeSongArrangement':
                    ThemeSongArrangement.append(job[0])
                elif profes == 'ThemeSongComposition':
                    ThemeSongComposition.append(job[0])
                elif profes == 'AnimationDirector':
                    AnimationDirector.append(job[0])
                elif profes == 'OriginalCreator':
                    OriginalCreator.append(job[0])
                elif profes == 'ADRDirector':
                    ADRDirector.append(job[0])
                elif profes == 'ExecutiveProducer':
                    ExecutiveProducer.append(job[0])
                elif profes == 'ChiefAnimationDirector':
                    ChiefAnimationDirector.append(job[0])
                elif profes == 'DirectorofPhotography':
                    DirectorofPhotography.append(job[0])
                elif profes == 'In-BetweenAnimation':
                    In_BetweenAnimation.append(job[0])
                elif profes == 'BackgroundArt':
                    BackgroundArt.append(job[0])
                elif profes == 'Planning':
                    Planning.append(job[0])
                elif profes == 'ColorDesign':
                    ColorDesign.append(job[0])
                elif profes == 'AssistantAnimationDirector':
                    AssistantAnimationDirector.append(job[0])
                elif profes == 'Editing':
                    Editing.append(job[0])
                elif profes == 'ArtDirector':
                    ArtDirector.append(job[0])
                elif profes == 'Music':
                    Music.append(job[0])
                elif profes == 'ProductionCoordination':
                    ProductionCoordination.append(job[0])
                elif profes == 'DigitalPaint':
                    DigitalPaint.append(job[0])
                elif profes == 'ColorSetting':
                    ColorSetting.append(job[0])
                elif profes == 'AssistantDirector':
                    AssistantDirector.append(job[0])
                elif profes == 'SoundEffects':
                    SoundEffects.append(job[0])
                elif profes == 'InsertedSongPerformance':
                    InsertedSongPerformance.append(job[0])
                elif profes == 'OriginalCharacterDesign':
                    OriginalCharacterDesign.append(job[0])
                elif profes == 'Setting':
                    Setting.append(job[0])
                elif profes == 'Screenplay':
                    Screenplay.append(job[0])
                elif profes == 'AssociateProducer':
                    AssociateProducer.append(job[0])
                elif profes == 'SpecialEffects':
                    SpecialEffects.append(job[0])
                elif profes == 'AnimationCheck':
                    AnimationCheck.append(job[0])
                elif profes == 'ProductionAssistant':
                    ProductionAssistant.append(job[0])
                elif profes == 'SoundSupervisor':
                    SoundSupervisor.append(job[0])
                elif profes == 'PrincipleDrawing':
                    PrincipleDrawing.append(job[0])
                elif profes == 'AssistantProducer':
                    AssistantProducer.append(job[0])
                elif profes == 'ProductionManager':
                    ProductionManager.append(job[0])
                elif profes == 'SoundManager':
                    SoundManager.append(job[0])
                elif profes == 'Co-Director':
                    Co_Director.append(job[0])
                elif profes == 'SeriesProductionDirector':
                    SeriesProductionDirector.append(job[0])
                elif profes == 'MechanicalDesign':
                    MechanicalDesign.append(job[0])
                elif profes == 'PlanningProducer':
                    PlanningProducer.append(job[0])
                elif profes == 'Layout':
                    Layout.append(job[0])
                elif profes == 'ChiefProducer':
                    ChiefProducer.append(job[0])
                elif profes == 'Recording':
                    Recording.append(job[0])
                elif profes == 'DialogueEditing':
                    DialogueEditing.append(job[0])
                elif profes == 'RecordingEngineer':
                    RecordingEngineer.append(job[0])
                elif profes == 'Creator':
                    Creator.append(job[0])
                elif profes == 'RecordingAssistant':
                    RecordingAssistant.append(job[0])
                elif profes == 'Publicity':
                    Publicity.append(job[0])
                elif profes == 'Co-Producer':
                    Co_Producer.append(job[0])
                elif profes == 'CastingDirector':
                    CastingDirector.append(job[0])
                elif profes == 'Re-RecordingMixing':
                    Re_RecordingMixing.append(job[0])
                elif profes == 'AssistantProductionCoordinat':
                    AssistantProductionCoordinat.append(job[0])
                elif profes == 'SettingManager':
                    SettingManager.append(job[0])
    
    # Adicionar listas preenchidas ao dicionário
    profes_dic["Producer"].append(";".join(Producer))
    profes_dic["Director"].append(";".join(Director))
    profes_dic["EpisodeDirector"].append(";".join(EpisodeDirector))
    profes_dic["Storyboard"].append(";".join(Storyboard))
    profes_dic["SoundDirector"].append(";".join(SoundDirector))
    profes_dic["KeyAnimation"].append(";".join(KeyAnimation))
    profes_dic["2ndKeyAnimation"].append(";".join(KeyAnimation2nd))
    profes_dic["CharacterDesign"].append(";".join(CharacterDesign))
    profes_dic["Script"].append(";".join(Script))
    profes_dic["SeriesComposition"].append(";".join(SeriesComposition))
    profes_dic["ThemeSongLyrics"].append(";".join(ThemeSongLyrics))
    profes_dic["ThemeSongPerformance"].append(";".join(ThemeSongPerformance))
    profes_dic["ThemeSongArrangement"].append(";".join(ThemeSongArrangement))
    profes_dic["ThemeSongComposition"].append(";".join(ThemeSongComposition))
    profes_dic["AnimationDirector"].append(";".join(AnimationDirector))
    profes_dic["OriginalCreator"].append(";".join(OriginalCreator))
    profes_dic["ADRDirector"].append(";".join(ADRDirector))
    profes_dic["ExecutiveProducer"].append(";".join(ExecutiveProducer))
    profes_dic["ChiefAnimationDirector"].append(";".join(ChiefAnimationDirector))
    profes_dic["DirectorofPhotography"].append(";".join(DirectorofPhotography))
    profes_dic["In-BetweenAnimation"].append(";".join(In_BetweenAnimation))
    profes_dic["BackgroundArt"].append(";".join(BackgroundArt))
    profes_dic["Planning"].append(";".join(Planning))
    profes_dic["ColorDesign"].append(";".join(ColorDesign))
    profes_dic["AssistantAnimationDirector"].append(";".join(AssistantAnimationDirector))
    profes_dic["Editing"].append(";".join(Editing))
    profes_dic["ArtDirector"].append(";".join(ArtDirector))
    profes_dic["Music"].append(";".join(Music))
    profes_dic["ProductionCoordination"].append(";".join(ProductionCoordination))
    profes_dic["DigitalPaint"].append(";".join(DigitalPaint))
    profes_dic["ColorSetting"].append(";".join(ColorSetting))
    profes_dic["AssistantDirector"].append(";".join(AssistantDirector))
    profes_dic["SoundEffects"].append(";".join(SoundEffects))
    profes_dic["InsertedSongPerformance"].append(";".join(InsertedSongPerformance))
    profes_dic["OriginalCharacterDesign"].append(";".join(OriginalCharacterDesign))
    profes_dic["Setting"].append(";".join(Setting))
    profes_dic["Screenplay"].append(";".join(Screenplay))
    profes_dic["AssociateProducer"].append(";".join(AssociateProducer))
    profes_dic["SpecialEffects"].append(";".join(SpecialEffects))
    profes_dic["AnimationCheck"].append(";".join(AnimationCheck))
    profes_dic["ProductionAssistant"].append(";".join(ProductionAssistant))
    profes_dic["SoundSupervisor"].append(";".join(SoundSupervisor))
    profes_dic["PrincipleDrawing"].append(";".join(PrincipleDrawing))
    profes_dic["AssistantProducer"].append(";".join(AssistantProducer))
    profes_dic["ProductionManager"].append(";".join(ProductionManager))
    profes_dic["SoundManager"].append(";".join(SoundManager))
    profes_dic["Co-Director"].append(";".join(Co_Director))
    profes_dic["SeriesProductionDirector"].append(";".join(SeriesProductionDirector))
    profes_dic["MechanicalDesign"].append(";".join(MechanicalDesign))
    profes_dic["PlanningProducer"].append(";".join(PlanningProducer))
    profes_dic["Layout"].append(";".join(Layout))
    profes_dic["ChiefProducer"].append(";".join(ChiefProducer))
    profes_dic["Recording"].append(";".join(Recording))
    profes_dic["DialogueEditing"].append(";".join(DialogueEditing))
    profes_dic["RecordingEngineer"].append(";".join(RecordingEngineer))
    profes_dic["Creator"].append(";".join(Creator))
    profes_dic["RecordingAssistant"].append(";".join(RecordingAssistant))
    profes_dic["Publicity"].append(";".join(Publicity))
    profes_dic["Co-Producer"].append(";".join(Co_Producer))
    profes_dic["CastingDirector"].append(";".join(CastingDirector))
    profes_dic["Re-RecordingMixing"].append(";".join(Re_RecordingMixing))
    profes_dic["AssistantProductionCoordinat"].append(";".join(AssistantProductionCoordinat))
    profes_dic["SettingManager"].append(";".join(SettingManager))


In [300]:
len(profes_dic)

63

In [301]:
occupation = pd.DataFrame(profes_dic)

In [303]:
occupation.head(10)

,Producer,Director,EpisodeDirector,Storyboard,SoundDirector,KeyAnimation,2ndKeyAnimation,CharacterDesign,Script,SeriesComposition,ThemeSongLyrics,ThemeSongPerformance,ThemeSongArrangement,ThemeSongComposition,AnimationDirector,OriginalCreator,ADRDirector,ExecutiveProducer,ChiefAnimationDirector,DirectorofPhotography,In-BetweenAnimation,BackgroundArt,Planning,ColorDesign,AssistantAnimationDirector,Editing,ArtDirector,Music,ProductionCoordination,DigitalPaint,ColorSetting,AssistantDirector,SoundEffects,InsertedSongPerformance,OriginalCharacterDesign,Setting,Screenplay,AssociateProducer,SpecialEffects,AnimationCheck,ProductionAssistant,SoundSupervisor,PrincipleDrawing,AssistantProducer,ProductionManager,SoundManager,Co-Director,SeriesProductionDirector,MechanicalDesign,PlanningProducer,Layout,ChiefProducer,Recording,DialogueEditing,RecordingEngineer,Creator,RecordingAssistant,Publicity,Co-Producer,CastingDirector,Re-RecordingMixing,AssistantProductionCoordinat,SettingManager
0,"Aoki,Haruka;Fukushi,Yuuichirou;Itou,Yuukou;Iwa...","Saitou,Keiichirou","Saitou,Keiichirou;Andou,Ken;Asaka,Morio;Fujii,...","Saitou,Keiichirou;Fujimoto,Kouki;Harashina,Dai...","Hata,Shouji","Saitou,Keiichirou;Fujimoto,Kouki;Iwazawa,Tooru...","Saitou,Keiichirou;Kakita,Hayato;An,MiKyoung;Ar...","Nagasawa,Reiko","Suzuki,Tomohiro","Suzuki,Tomohiro",Ayase;milet;n-buna,YOASOBI;Yorushika;milet,"Ayase;Call,Evan;n-buna","Ayase;Call,Evan;Nakamura,Daisuke;Nomura,Youich...","Harashina,Daiki;Kariya,Nobuhide;Seguchi,Izumi;...","Yamada,Kanehito","Lecordier,Louis;Lecordier,Brigitte;Marques,Gui...","Bizenjima,Mikito;Yamanaka,Kazutaka","Fujinaka,Yuri;Nagasawa,Reiko;Takase,Akiko","Fushihara,Akane","Kojima,Keisuke","Sugiyama,Shinji;Yoshioka,Seiko","Higashiyama,Atsushi;Miyake,Masanori;Oota,Keiji...","Oono,Harue","Kudou,Shinya","Kimura,Kashiko","Takagi,Sawako","Call,Evan",,,,,"Izumo,Noriko",milet,"Abe,Tsukasa",,,"Asahara,Momoko;Hirahara,Yuito;Norose,Yuri;Take...","Fushihara,Akane","Kawamoto,Yukiko",,,,,,,,,,,,"Takahashi,Atsushi;Takei,Katsuhiro",,,,,,,,,,,
1,"Cook,Justin;Maruyama,Hiroo;Yonai,Noritomo","Irie,Yasuhiro","Irie,Yasuhiro;Igarashi,Takuya;Ikehata,Hiroshi;...","Irie,Yasuhiro;Igarashi,Takuya;Ikezoe,Takahiro;...","Mima,Masafumi","Irie,Yasuhiro;Ikezoe,Takahiro;Kinutani,Soushi;...","Irie,Yasuhiro;Ikezoe,Takahiro;Satou,Kiyomitsu;...","Morii,Kenshirou;Kanno,Hiroki","Burgmeier,John;Medel,Jennifer;Minakami,Seishi;...","Oonogi,Hiroshi",CHEMISTRY;NICOTouchestheWalls;SCANDAL;SID;Suki...,"CHEMISTRY;Fukuhara,Miho;[""LilB"";NICOTouchesthe...","Itagaki,Yuusuke;Kameda,Seiji;Kawaguchi,Keita;N...","NICOTouchestheWalls;SID;SukimaSwitch;Tajika,Yu...","Aono,Atsushi;Furumata,Taichi;Hata,Tomoji;Horik...","Arakawa,Hiromu","Bezerra,Úrsula;Campos,Marcelo;Fantini,Silvana;...","Fukunaga,Gen","Fukuoka,Hidenori;Kameda,Yoshimichi;Kaneda,Naomi","Takei,Yoshiyuki;Furumoto,Mayuko","Kameda,Yoshimichi;Katou,Sou;Matsumoto,Saori;Na...","Kanehira,Kazushige;Nakajima,Mika;Ooizumi,Anna;...","Higashiyama,Atsushi;Minami,Masahiko;Ueda,Masuo","Nakao,Fusako","Ishino,Satoshi;Kobayashi,Yuu;Kobayashi,Masami;...","Sadamatsu,Gou","Satou,Takeshi","Senju,Akira","Une,Shinya","Inoue,Akiko;Uemura,Takako",,,,,,,,"Maeda,Toshihiro",,,,,,,,,,,,,,,,,,,,"Endou,Tetsuya;Uryuu,Kyouko",,,,,
2,"Cook,Justin;Danno,Yoshito;Doi,Yoshinao;Gomi,Ke...","Hamasaki,Hiroshi;Satou,Takuya","Hamasaki,Hiroshi;Satou,Takuya;Hiramuki,Tomoko;...","Hamasaki,Hiroshi;Satou,Takuya;Ikeda,Shigetaka;...","Fujiyama,Fusanobu","Hamasaki,Hiroshi;Wakabayashi,Kanji;Aoki,Yuu;Ao...","Aoki,Yuu;Hamaguchi,Akira;Kurose,Daisuke;Mutou,...","Sakai,Kyuuta","Hanada,Jukki;Nemoto,Toshizou;Tatum,J.Michael;Y...","Hanada,Jukki","Shikura,Chiyomaru","Itou,Kanako;Sakakibara,Yui","Abo,Takeshi","Abo,Takeshi;Shikura,Chiyomaru","Aono,Atsushi;Arai,Nobuhiro;Igari,Takashi;Ikega...",5pb.;Nitroplus,"Clinkenbeard,Colleen;Dawn-Claude,René;Dutel,Ju...","Fukunaga,Gen","Sakai,Kyuuta","Nakamura,Keisuke","Aoki,Yuu;Busa,Yukiko;Masuda,Yuusa;Murai,Kaori;...","Etou,Kouji;Kanehira,Kazushige;Kim,HyunSoo;Maed..."

In [304]:
df1 = df.merge(occupation, left_index = True, right_index = True, sort = False)

In [305]:
df1.head()

,name,producers,ranking,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,voiceActorNameandLanguage,staffNameandOccupation,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew,Producer,Director,EpisodeDirector,Storyboard,SoundDirector,KeyAnimation,2ndKeyAnimation,CharacterDesign,Script,SeriesComposition,ThemeSongLyrics,ThemeSongPerformance,ThemeSongArrangement,ThemeSongComposition,AnimationDirector,OriginalCreator,ADRDirector,ExecutiveProducer,ChiefAnimationDirector,DirectorofPhotography,In-BetweenAnimation,BackgroundArt,Planning,ColorDesign,AssistantAnimationDirector,Editing,ArtDirector,Music,ProductionCoordination,DigitalPaint,ColorSetting,AssistantDirector,SoundEffects,InsertedSongPerformance,OriginalCharacterDesign,Setting,Screenplay,AssociateProducer,SpecialEffects,AnimationCheck,ProductionAssistant,SoundSupervisor,PrincipleDrawing,AssistantProducer,ProductionManager,SoundManager,Co-Director,SeriesProductionDirector,MechanicalDesign,PlanningProducer,Layout,ChiefProducer,Recording,DialogueEditing,RecordingEngineer,Creator,RecordingAssistant,Publicity,Co-Producer,CastingDirector,Re-RecordingMixing,AssistantProductionCoordinat,SettingManager
0,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,1,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"[['Ichinose,Kana', 'Japanese'], ['Harris,Jill'...","[['Aoki,Haruka', 'Producer'], ['Fukushi,Yuuich...","Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",,,,,"Aoki,Haruka;Fukushi,Yuuichirou;Itou,Yuukou;Iwa...","Saitou,Keiichirou","Saitou,Keiichirou;Andou,Ken;Asaka,Morio;Fujii,...","Saitou,Keiichirou;Fujimoto,Kouki;Harashina,Dai...","Hata,Shouji","Saitou,Keiichirou;Fujimoto,Kouki;Iwazawa,Tooru...","Saitou,Keiichirou;Kakita,Hayato;An,MiKyoung;Ar...","Nagasawa,Reiko","Suzuki,Tomohiro","Suzuki,Tomohiro",Ayase;milet;n-buna,YOASOBI;Yorushika;milet,"Ayase;Call,Evan;n-buna","Ayase;Call,Evan;Nakamura,Daisuke;Nomura,Youich...","Harashina,Daiki;Kariya,Nobuhide;Seguchi,Izumi;...","Yamada,Kanehito","Lecordier,Louis;Lecordier,Brigitte;Marques,Gui...","Bizenjima,Mikito;Yamanaka,Kazutaka","Fujinaka,Yuri;Nagasawa,Reiko;Takase,Akiko","Fushihara,Akane","Kojima,Keisuke","Sugiyama,Shinji;Yoshioka,Seiko","Higashiyama,Atsushi;Miyake,Masanori;Oota,Keiji...","Oono,Harue","Kudou,Shinya","Kimura,Kashiko","Takagi,Sawako","Call,Evan",,,,,"Izumo,Noriko",milet,"Abe,Tsukasa",,,"Asahara,Momoko;Hirahara,Yuito;Norose,Yuri;Take...","Fushihara,Akane","Kawamoto,Yukiko",,,,,,,,,,,,"Takahashi,Atsushi;Takei,Katsuhiro",,,,,,,,,,,
1,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"[['Kugimiya,Rie', 'Japanese'], ['Ponticelli,Be...","[['Cook,Justin', 'Producer'], ['Maruyama,Hiroo...","Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",,,"Cook,Justin;Maruyama,Hiroo;Yonai,Noritomo","Irie,Yasuhiro","Irie,Yasuhiro;Igarashi,Takuya;Ikehata,Hiroshi;...","Irie,Yasuhiro;Igarashi,Takuya;Ikezoe,Takahiro;...","Mima,Masafumi","Irie,Yasuhiro;Ikezoe,Takahiro;Kinutani,Soushi;...","Irie,Yas

In [306]:
df1.drop(['voiceActorNameandLanguage', 'staffNameandOccupation'], axis = 1, inplace = True)

In [307]:
# Reindexação do dataframe
df1.set_index("ranking", inplace = True)

In [308]:
df1.head()

,name,producers,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew,Producer,Director,EpisodeDirector,Storyboard,SoundDirector,KeyAnimation,2ndKeyAnimation,CharacterDesign,Script,SeriesComposition,ThemeSongLyrics,ThemeSongPerformance,ThemeSongArrangement,ThemeSongComposition,AnimationDirector,OriginalCreator,ADRDirector,ExecutiveProducer,ChiefAnimationDirector,DirectorofPhotography,In-BetweenAnimation,BackgroundArt,Planning,ColorDesign,AssistantAnimationDirector,Editing,ArtDirector,Music,ProductionCoordination,DigitalPaint,ColorSetting,AssistantDirector,SoundEffects,InsertedSongPerformance,OriginalCharacterDesign,Setting,Screenplay,AssociateProducer,SpecialEffects,AnimationCheck,ProductionAssistant,SoundSupervisor,PrincipleDrawing,AssistantProducer,ProductionManager,SoundManager,Co-Director,SeriesProductionDirector,MechanicalDesign,PlanningProducer,Layout,ChiefProducer,Recording,DialogueEditing,RecordingEngineer,Creator,RecordingAssistant,Publicity,Co-Producer,CastingDirector,Re-RecordingMixing,AssistantProductionCoordinat,SettingManager
ranking,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",,,,,"Aoki,Haruka;Fukushi,Yuuichirou;Itou,Yuukou;Iwa...","Saitou,Keiichirou","Saitou,Keiichirou;Andou,Ken;Asaka,Morio;Fujii,...","Saitou,Keiichirou;Fujimoto,Kouki;Harashina,Dai...","Hata,Shouji","Saitou,Keiichirou;Fujimoto,Kouki;Iwazawa,Tooru...","Saitou,Keiichirou;Kakita,Hayato;An,MiKyoung;Ar...","Nagasawa,Reiko","Suzuki,Tomohiro","Suzuki,Tomohiro",Ayase;milet;n-buna,YOASOBI;Yorushika;milet,"Ayase;Call,Evan;n-buna","Ayase;Call,Evan;Nakamura,Daisuke;Nomura,Youich...","Harashina,Daiki;Kariya,Nobuhide;Seguchi,Izumi;...","Yamada,Kanehito","Lecordier,Louis;Lecordier,Brigitte;Marques,Gui...","Bizenjima,Mikito;Yamanaka,Kazutaka","Fujinaka,Yuri;Nagasawa,Reiko;Takase,Akiko","Fushihara,Akane","Kojima,Keisuke","Sugiyama,Shinji;Yoshioka,Seiko","Higashiyama,Atsushi;Miyake,Masanori;Oota,Keiji...","Oono,Harue","Kudou,Shinya","Kimura,Kashiko","Takagi,Sawako","Call,Evan",,,,,"Izumo,Noriko",milet,"Abe,Tsukasa",,,"Asahara,Momoko;Hirahara,Yuito;Norose,Yuri;Take...","Fushihara,Akane","Kawamoto,Yukiko",,,,,,,,,,,,"Takahashi,Atsushi;Takei,Katsuhiro",,,,,,,,,,,
2,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",,,"Cook,Justin;Maruyama,Hiroo;Yonai,Noritomo","Irie,Yasuhiro","Irie,Yasuhiro;Igarashi,Takuya;Ikehata,Hiroshi;...","Irie,Yasuhiro;Igarashi,Takuya;Ikezoe,Takahiro;...","Mima,Masafumi","Irie,Yasuhiro;Ikezoe,Takahiro;Kinutani,Soushi;...","Irie,Yasuhiro;Ikezoe,Takahiro;Satou,Kiyomitsu;...","Morii,Kenshirou;Kanno,Hiroki","Burgmeier,John;Medel,Jennifer;Minakami,Seishi;...","Oonogi,Hiroshi",CHEMISTRY;NICOTouchestheWalls

In [310]:
df1.to_csv("/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/Processamento/animeData.csv")

In [312]:
dfFinal = pd.read_csv("/Users/duda/PythonDSA/Duda/8-Projeto7/Projeto7/dados/Processamento/animeData.csv")

In [313]:
pd.set_option("display.max_columns", 97)

In [314]:
dfFinal.head()

,ranking,name,producers,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew,Producer,Director,EpisodeDirector,Storyboard,SoundDirector,KeyAnimation,2ndKeyAnimation,CharacterDesign,Script,SeriesComposition,ThemeSongLyrics,ThemeSongPerformance,ThemeSongArrangement,ThemeSongComposition,AnimationDirector,OriginalCreator,ADRDirector,ExecutiveProducer,ChiefAnimationDirector,DirectorofPhotography,In-BetweenAnimation,BackgroundArt,Planning,ColorDesign,AssistantAnimationDirector,Editing,ArtDirector,Music,ProductionCoordination,DigitalPaint,ColorSetting,AssistantDirector,SoundEffects,InsertedSongPerformance,OriginalCharacterDesign,Setting,Screenplay,AssociateProducer,SpecialEffects,AnimationCheck,ProductionAssistant,SoundSupervisor,PrincipleDrawing,AssistantProducer,ProductionManager,SoundManager,Co-Director,SeriesProductionDirector,MechanicalDesign,PlanningProducer,Layout,ChiefProducer,Recording,DialogueEditing,RecordingEngineer,Creator,RecordingAssistant,Publicity,Co-Producer,CastingDirector,Re-RecordingMixing,AssistantProductionCoordinat,SettingManager
0,1,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,434218,9.35,247,783323,45390,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",NaN,NaN,NaN,NaN,"Aoki,Haruka;Fukushi,Yuuichirou;Itou,Yuukou;Iwa...","Saitou,Keiichirou","Saitou,Keiichirou;Andou,Ken;Asaka,Morio;Fujii,...","Saitou,Keiichirou;Fujimoto,Kouki;Harashina,Dai...","Hata,Shouji","Saitou,Keiichirou;Fujimoto,Kouki;Iwazawa,Tooru...","Saitou,Keiichirou;Kakita,Hayato;An,MiKyoung;Ar...","Nagasawa,Reiko","Suzuki,Tomohiro","Suzuki,Tomohiro",Ayase;milet;n-buna,YOASOBI;Yorushika;milet,"Ayase;Call,Evan;n-buna","Ayase;Call,Evan;Nakamura,Daisuke;Nomura,Youich...","Harashina,Daiki;Kariya,Nobuhide;Seguchi,Izumi;...","Yamada,Kanehito","Lecordier,Louis;Lecordier,Brigitte;Marques,Gui...","Bizenjima,Mikito;Yamanaka,Kazutaka","Fujinaka,Yuri;Nagasawa,Reiko;Takase,Akiko","Fushihara,Akane","Kojima,Keisuke","Sugiyama,Shinji;Yoshioka,Seiko","Higashiyama,Atsushi;Miyake,Masanori;Oota,Keiji...","Oono,Harue","Kudou,Shinya","Kimura,Kashiko","Takagi,Sawako","Call,Evan",NaN,NaN,NaN,NaN,"Izumo,Noriko",milet,"Abe,Tsukasa",NaN,NaN,"Asahara,Momoko;Hirahara,Yuito;Norose,Yuri;Take...","Fushihara,Akane","Kawamoto,Yukiko",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Takahashi,Atsushi;Takei,Katsuhiro",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2133065,9.09,3,3366848,226746,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",NaN,NaN,"Cook,Justin;Maruyama,Hiroo;Yonai,Noritomo","Irie,Yasuhiro","Irie,Yasuhiro;Igarashi,Takuya;Ikehata,Hiroshi;...","Irie,Yasuhiro;Igarashi,Takuya;Ikezoe,Takahiro;...","Mima,Masafumi","Irie,Yasuhiro;Ikezoe,Takahiro;Kinutani,Soushi;...","Irie,Yasuhiro;Ikezoe,Takahiro;Satou,Kiyomitsu;...","Morii,Kenshirou;Kanno,Hiroki","Burgmeier,John;Medel,Jennifer;Minakami,Seishi;...","Oonogi,Hiroshi",CHEMISTRY;NI

In [316]:
dfFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 90 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ranking                       500 non-null    int64  
 1   name                          500 non-null    object 
 2   producers                     500 non-null    object 
 3   votersCount                   500 non-null    int64  
 4   score                         500 non-null    float64
 5   popularity                    500 non-null    int64  
 6   members                       500 non-null    int64  
 7   favorite                      500 non-null    int64  
 8   episodes                      497 non-null    float64
 9   animeType                     500 non-null    object 
 10  premiredDate                  301 non-null    object 
 11  animeSource                   500 non-null    object 
 12  genres                        404 non-null    object 
 13  studi

In [317]:
dfFinal.columns

Index(['ranking', 'name', 'producers', 'votersCount', 'score', 'popularity',
       'members', 'favorite', 'episodes', 'animeType', 'premiredDate',
       'animeSource', 'genres', 'studios', 'licenses', 'rating', 'Japanese',
       'English', 'French', 'Portuguese(BR)', 'Italian', 'Spanish', 'German',
       'Hungarian', 'Korean', 'Mandarin', 'Hebrew', 'Producer', 'Director',
       'EpisodeDirector', 'Storyboard', 'SoundDirector', 'KeyAnimation',
       '2ndKeyAnimation', 'CharacterDesign', 'Script', 'SeriesComposition',
       'ThemeSongLyrics', 'ThemeSongPerformance', 'ThemeSongArrangement',
       'ThemeSongComposition', 'AnimationDirector', 'OriginalCreator',
       'ADRDirector', 'ExecutiveProducer', 'ChiefAnimationDirector',
       'DirectorofPhotography', 'In-BetweenAnimation', 'BackgroundArt',
       'Planning', 'ColorDesign', 'AssistantAnimationDirector', 'Editing',
       'ArtDirector', 'Music', 'ProductionCoordination', 'DigitalPaint',
       'ColorSetting', 'AssistantDirec

In [320]:
# 1- Qual a média de score dos animes?

dfFinal.score.isna().any()

False

In [321]:
dfFinal.score.mean()

8.381780000000001

#### Questões

In [332]:
# 2- Qual o anime com maior número de votos?

dfFinal.votersCount.max()

2814322

In [333]:
dfFinal.loc[dfFinal['votersCount'] == 2814322, 'name']

105    Shingeki no Kyojin
Name: name, dtype: object

In [334]:
# 3- Quais os tipos de animes?

dfFinal.animeType.value_counts()

animeType
TV         313
Movie      117
ONA         33
OVA         23
Special     14
Name: count, dtype: int64

In [336]:
# 4- Qual a média de score por tipo de anime?

dfFinal.groupby(['animeType'])['score'].mean()

animeType
Movie      8.362051
ONA        8.350606
OVA        8.300000
Special    8.261429
TV         8.403834
Name: score, dtype: float64

In [338]:
# 5- Quais são as fontes dos animes?

dfFinal.animeSource.value_counts()

animeSource
Manga           289
Original         65
Light novel      54
Novel            33
Web manga        19
Visual novel     14
Web novel        11
4-koma manga      4
Game              4
Other             3
Book              1
Unknown           1
Mixed media       1
Music             1
Name: count, dtype: int64

In [340]:
# 6- Quantos animes estão disponíveis em português?

dfFinal['Portuguese(BR)'].value_counts().sum()

191

In [344]:
# 7- Quantos animes têm co-produtores (Co-Producer)?

dfFinal['Co-Producer'].value_counts().sum()

5

In [347]:
# 8- Há animes sem produtores?

dfFinal.producers.isna().any()

False

In [349]:
dfFinal.producers.value_counts()

producers
Nonefound;addsome                                                          37
Aniplex                                                                     8
Shogakukan-ShueishaProductions;NHK                                          6
TencentPenguinPictures                                                      6
VAP;NipponTelevisionNetwork                                                 6
                                                                           ..
BandaiVisual;Lantis;Rakuonsha;Showgate;HakuhodoDYMusic&Pictures;Cygames     1
Shochiku;Kadokawa;Sonilude;MovieWalker                                      1
TVTokyo;TVOsaka;Funimation;NihonAdSystems;Hakusensha;AvexPictures;8PAN      1
TencentPenguinPictures;ChinaLiteratureLimited;WandaMedia                    1
BandaiVisual;NihonAdSystems                                                 1
Name: count, Length: 344, dtype: int64

In [350]:
dfFinal[dfFinal["producers"].str.contains("Nonefound")] = np.nan

In [353]:
dfFinal.producers.isna().sum()

37

In [356]:
# 9- Quantos vezes cada produtor participou da produção de um anime?

unique_producers = []

for data in df["producers"]:
    if type(data) != float:
        new_data = data.split(";")
        for producer in new_data:
            if producer not in unique_producers:
                unique_producers.append(producer)

In [357]:
unique_producers

['Aniplex',
 'Dentsu',
 'Shogakukan-ShueishaProductions',
 'NipponTelevisionNetwork',
 'TOHOanimation',
 'Shogakukan',
 'SquareEnix',
 'MainichiBroadcastingSystem',
 'StudioMoriken',
 'FrontierWorks',
 'MediaFactory',
 'KadokawaShoten',
 'Movic',
 'AT-X',
 'KadokawaPicturesJapan',
 'Nitroplus',
 'TVTokyo',
 'ProductionI.G',
 'PonyCanyon',
 'Kodansha',
 'TechnoSound',
 'PonyCanyonEnterprises',
 'Bandai',
 'WarnerBros.Japan',
 'Shueisha',
 'TrinitySound',
 'MiracleRobo',
 'StudioJack',
 'VAP',
 'KittyFilms',
 'TokumaShoten',
 'TokumaJapanCommunications',
 'WrightStaff',
 'MiracleBus',
 'MagicCapsule',
 'JREastMarketing&Communications',
 'TVOsaka',
 'NihonAdSystems',
 'Hakusensha',
 'AvexPictures',
 'TMSMusic',
 '8PAN',
 'AudioHighs',
 'TBS',
 'Rakuonsha',
 'AnimationDo',
 'Shochiku',
 'ABCAnimation',
 'Quaras',
 'NHK',
 'AsmikAce',
 '"ToysFactory"',
 'BandaiVisual',
 'BandaiNamcoEntertainment',
 'HakuhodoDYMediaPartners',
 'ImagicaInfos',
 'TOHO',
 'SoundTeamDonJuan',
 'LawsonHMVEntertai

In [358]:
prod_dic = dict()

In [359]:
for i in unique_producers:
    prod_dic[i] = 0

In [360]:
for prod in unique_producers:
    for i in range(0, len(df["producers"])):
        if type(df["producers"][i]) != float and (df["producers"][i].__contains__(prod)) == True:
            prod_dic[prod] += 1

In [361]:
prod_dic

{'Aniplex': 104,
 'Dentsu': 77,
 'Shogakukan-ShueishaProductions': 24,
 'NipponTelevisionNetwork': 30,
 'TOHOanimation': 32,
 'Shogakukan': 34,
 'SquareEnix': 11,
 'MainichiBroadcastingSystem': 47,
 'StudioMoriken': 1,
 'FrontierWorks': 14,
 'MediaFactory': 6,
 'KadokawaShoten': 12,
 'Movic': 59,
 'AT-X': 29,
 'KadokawaPicturesJapan': 6,
 'Nitroplus': 10,
 'TVTokyo': 53,
 'ProductionI.G': 13,
 'PonyCanyon': 31,
 'Kodansha': 61,
 'TechnoSound': 4,
 'PonyCanyonEnterprises': 9,
 'Bandai': 51,
 'WarnerBros.Japan': 18,
 'Shueisha': 93,
 'TrinitySound': 4,
 'MiracleRobo': 2,
 'StudioJack': 2,
 'VAP': 21,
 'KittyFilms': 3,
 'TokumaShoten': 11,
 'TokumaJapanCommunications': 1,
 'WrightStaff': 4,
 'MiracleBus': 1,
 'MagicCapsule': 12,
 'JREastMarketing&Communications': 11,
 'TVOsaka': 4,
 'NihonAdSystems': 15,
 'Hakusensha': 16,
 'AvexPictures': 15,
 'TMSMusic': 5,
 '8PAN': 6,
 'AudioHighs': 1,
 'TBS': 6,
 'Rakuonsha': 11,
 'AnimationDo': 5,
 'Shochiku': 17,
 'ABCAnimation': 11,
 'Quaras': 1,
 

In [364]:
# Ordena o resultado

{k: v for k, v in sorted(prod_dic.items(), key = lambda item: item[1])}

{'StudioMoriken': 1,
 'TokumaJapanCommunications': 1,
 'MiracleBus': 1,
 'AudioHighs': 1,
 'Quaras': 1,
 'ImagicaInfos': 1,
 'Amuse': 1,
 'Mitsubishi': 1,
 'CycloneGraphics': 1,
 'JinnanStudio': 1,
 'TAP': 1,
 'Annapuru': 1,
 'ZackPromotion': 1,
 'YoshimotoKogyo': 1,
 'CyberAgent': 1,
 'Konami': 1,
 'CDProjektRed': 1,
 'TencentJapan': 1,
 'Madhouse': 1,
 'HiroshimaTelevision': 1,
 'Sega': 1,
 'RexEntertainment': 1,
 'ONIRO': 1,
 'DocomoAnimeStore': 1,
 'BarnumStudio': 1,
 'TezukaProductions': 1,
 'CA-CygamesAnimeFund': 1,
 'TCEntertainment': 1,
 'NipponTelevisionMusic': 1,
 'FBC': 1,
 'SpaceyMusicEntertainment': 1,
 'TrilogyFutureStudio': 1,
 'NAZ': 1,
 'WandaMedia': 1,
 'Funimation': 1,
 'YomiuriShimbun': 1,
 'MediaLinkEntertainmentLimited': 1,
 'TEMCo.': 1,
 'Ltd.': 1,
 'NiceBoatAnimation': 1,
 'GeneonEntertainmentUSA': 1,
 'ShounenGahousha': 1,
 'Khara': 1,
 'Shodensha': 1,
 'ForecastCommunications': 1,
 'ADKMarketingSolutions': 1,
 'OLM': 1,
 'VoyagerEntertainment': 1,
 'Netmarble'

In [367]:
prod_dic["Aniplex"]

104

In [369]:
prod_dic["Shueisha"]

93

In [368]:
prod_dic["Dentsu"]

77

In [370]:
# 10- Considerando o produtor que mais participou da produção de animes, qual a média de score, popularidade e membros?

df2 = dfFinal.dropna(subset = ['producers'])

In [371]:
df3 = df2[df2["producers"].str.contains("Aniplex")]

In [373]:
df3.shape

(104, 90)

In [372]:
df3.head()

,ranking,name,producers,votersCount,score,popularity,members,favorite,episodes,animeType,premiredDate,animeSource,genres,studios,licenses,rating,Japanese,English,French,Portuguese(BR),Italian,Spanish,German,Hungarian,Korean,Mandarin,Hebrew,Producer,Director,EpisodeDirector,Storyboard,SoundDirector,KeyAnimation,2ndKeyAnimation,CharacterDesign,Script,SeriesComposition,ThemeSongLyrics,ThemeSongPerformance,ThemeSongArrangement,ThemeSongComposition,AnimationDirector,OriginalCreator,ADRDirector,ExecutiveProducer,ChiefAnimationDirector,DirectorofPhotography,In-BetweenAnimation,BackgroundArt,Planning,ColorDesign,AssistantAnimationDirector,Editing,ArtDirector,Music,ProductionCoordination,DigitalPaint,ColorSetting,AssistantDirector,SoundEffects,InsertedSongPerformance,OriginalCharacterDesign,Setting,Screenplay,AssociateProducer,SpecialEffects,AnimationCheck,ProductionAssistant,SoundSupervisor,PrincipleDrawing,AssistantProducer,ProductionManager,SoundManager,Co-Director,SeriesProductionDirector,MechanicalDesign,PlanningProducer,Layout,ChiefProducer,Recording,DialogueEditing,RecordingEngineer,Creator,RecordingAssistant,Publicity,Co-Producer,CastingDirector,Re-RecordingMixing,AssistantProductionCoordinat,SettingManager
0,1.0,Sousou no Frieren,Aniplex;Dentsu;Shogakukan-ShueishaProductions;...,434218.0,9.35,247.0,783323.0,45390.0,28.0,TV,Fall 2023,Manga,Adventure;Drama;Fantasy,Madhouse,NaN,PG-13-Teens,"Ichinose,Kana;Tanezaki,Atsumi;Kobayashi,Chiaki...","Harris,Jill;Rodak,Mallorie;Cruz,JordanDash;Kar...","Thellier,Jehanne;Nonnenmacher,Marie;Lecordier,...","Rosis,MariaClara;Souza,Jacque;Miagusuku,Lucas;...","Marteddu,Agnese;Felli,Martina;Marteddu,Tito;Ru...","Estrada,Azucena;Ugalde,Erika;MartínezTorizes,E...","Fölster,Linda;Casper,Julia;Schächter,Janek;Vuc...",NaN,NaN,NaN,NaN,"Aoki,Haruka;Fukushi,Yuuichirou;Itou,Yuukou;Iwa...","Saitou,Keiichirou","Saitou,Keiichirou;Andou,Ken;Asaka,Morio;Fujii,...","Saitou,Keiichirou;Fujimoto,Kouki;Harashina,Dai...","Hata,Shouji","Saitou,Keiichirou;Fujimoto,Kouki;Iwazawa,Tooru...","Saitou,Keiichirou;Kakita,Hayato;An,MiKyoung;Ar...","Nagasawa,Reiko","Suzuki,Tomohiro","Suzuki,Tomohiro",Ayase;milet;n-buna,YOASOBI;Yorushika;milet,"Ayase;Call,Evan;n-buna","Ayase;Call,Evan;Nakamura,Daisuke;Nomura,Youich...","Harashina,Daiki;Kariya,Nobuhide;Seguchi,Izumi;...","Yamada,Kanehito","Lecordier,Louis;Lecordier,Brigitte;Marques,Gui...","Bizenjima,Mikito;Yamanaka,Kazutaka","Fujinaka,Yuri;Nagasawa,Reiko;Takase,Akiko","Fushihara,Akane","Kojima,Keisuke","Sugiyama,Shinji;Yoshioka,Seiko","Higashiyama,Atsushi;Miyake,Masanori;Oota,Keiji...","Oono,Harue","Kudou,Shinya","Kimura,Kashiko","Takagi,Sawako","Call,Evan",NaN,NaN,NaN,NaN,"Izumo,Noriko",milet,"Abe,Tsukasa",NaN,NaN,"Asahara,Momoko;Hirahara,Yuito;Norose,Yuri;Take...","Fushihara,Akane","Kawamoto,Yukiko",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Takahashi,Atsushi;Takei,Katsuhiro",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Fullmetal Alchemist: Brotherhood,Aniplex;SquareEnix;MainichiBroadcastingSystem;...,2133065.0,9.09,3.0,3366848.0,226746.0,64.0,TV,Spring 2009,Manga,Action;Adventure;Drama;Fantasy,Bones,Funimation;AniplexofAmerica,R-17+,"Kugimiya,Rie;Park,Romi;Soumi,Youko;Kugimiya,Ri...","Whitehead,Maxey;Truncale,Lily;Mignogna,Vic;Hof...","Pic,Audrey;Boudonnat,Lucille;Pestel,Arthur;Cré...","Andreatto,Rodrigo;Santana,Marina;Campos,Marcel...","Ponticelli,Benedetta;Novara,Renato;Scianca,Pat...","Torres,Jhonny;Vieira,JoséManuel;Escobar,Ricard...","Gerick,Wanja;Turba,David;Häcke,Maximiliane;Mai...","Baráth,István;Szabó,Máté;Faragó,András;Varga,R...","Yun,MiNa;O,InSeong;Yang,JeongHwa;Lee,JiYeong;H...",NaN,NaN,"Cook,Justin;Maruyama,Hiroo;Yonai,Noritomo","Irie,Yasuhiro","Irie,Yasuhiro;Igarashi,Takuya;Ikehata,Hiroshi;...","Irie,Yasuhiro;Igarashi,Takuya;Ikezoe,Takahiro;...","Mima,Masafumi","Irie,Yasuhiro;Ikezoe,Takahiro;Kinutani,Soushi;...","Irie,Yasuhiro;Ikezoe,Takahiro;Satou,Kiyomitsu;...","Morii,Kenshirou;Kanno,Hiroki","Burgmeier,John;Medel,Jennifer;Minakami,Seishi;...","Oonogi,H

In [374]:
df3['score'].mean()

8.50076923076923

In [375]:
df3['popularity'].mean()

671.8557692307693

In [376]:
df3['members'].mean()

706981.1634615385